In [3]:
import wandb
import pandas as pd


# 登录 wandb
wandb.login()

# 获取项目中的所有 run
api = wandb.Api()
path = './custom/checkpoint/verl-grpo_Qwen2.5-3B_max_response1280_batch48_rollout6_valbatch6_ppomini24_logprobbatch2_klcoef0.001_entcoef0.001_epochs3_simplelr_qwen_level3to5_origin_/wandb/offline-run-20250612_223740-wwer4aoy'
runs = api.runs(path)

wandb: Network error (ConnectTimeout), entering retry loop.


ConnectTimeout: HTTPSConnectionPool(host='api.wandb.ai', port=443): Max retries exceeded with url: /graphql (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79c95fd41f30>, 'Connection to api.wandb.ai timed out. (connect timeout=19)'))